## Get Data

In [1]:
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.33.0
    Uninstalling accelerate-0.33.0:
      Successfully uninstalled accelerate-0.33.0


In [2]:
!pip install --upgrade datasets transformers accelerate evaluate jiwer tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
ERROR: pip's dependency resolver

In [4]:
!pip install evaluate rouge_score transformers[torch]

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=5e9c9b27cda2e233750ab4a78e3984a49f0a4902c3c26c78a1d746a811fc77df
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [3]:
from datasets import load_dataset, DatasetDict, concatenate_datasets

In [4]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
os.chdir("drive/MyDrive/NLP Bootcamp/Final Project")
os.getcwd()

'/content/drive/MyDrive/NLP Bootcamp/Final Project'

In [ ]:
dataset = load_dataset("sehyun66/News-sentiments","distill",num_proc=4)

Setting num_proc from 4 back to 1 for the default split to disable multiprocessing as it only contains one shard.


Generating default split:   0%|          | 0/316086 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    default: Dataset({
        features: ['headline', 'summary', 'headline_sentiment', 'summary_sentiment'],
        num_rows: 316086
    })
})

In [ ]:
for row in dataset['default']:
  print(row)
  print(row['headline_sentiment'])
  print(row['summary_sentiment'])
  break

{'headline': 'Notable companies reporting after market close', 'summary': 'Looking for stock market analysis and research with proves results? Zacks.com offers in-depth financial research with over 30years of proven results.', 'headline_sentiment': {'postive': '0.25946614146232605', 'negative': '0.3799891471862793', 'neutral': '0.25364914536476135'}, 'summary_sentiment': {'postive': '0.2572893500328064', 'negative': '0.37214919924736023', 'neutral': '0.2266107201576233'}}
{'postive': '0.25946614146232605', 'negative': '0.3799891471862793', 'neutral': '0.25364914536476135'}
{'postive': '0.2572893500328064', 'negative': '0.37214919924736023', 'neutral': '0.2266107201576233'}


## Data Preparation

In [ ]:
import ast
def get_max_label(dict_label):
  dict_label = ast.literal_eval(str(dict_label))
  return max(dict_label, key=lambda k: float(dict_label[k]))

label_mapper = {
    'postive': 1,
    'neutral': 2,
    'negative': 0
}

def get_label(example):
    example['headline_label_name'] = get_max_label(example['headline_sentiment'])
    example['summary_label_name'] = get_max_label(example['summary_sentiment'])
    example['headline_label'] = label_mapper[example['headline_label_name']]
    example['summary_label'] = label_mapper[example['summary_label_name']]
    return example

In [ ]:
dataset['default'] = dataset['default'].map(get_label)

Map:   0%|          | 0/316086 [00:00<?, ? examples/s]

In [ ]:
for row in dataset['default']:
  print(row)
  print(row['headline_sentiment'])
  print(row['summary_sentiment'])
  print(row['headline_label'])
  print(row['summary_label'])
  break

{'headline': 'Notable companies reporting after market close', 'summary': 'Looking for stock market analysis and research with proves results? Zacks.com offers in-depth financial research with over 30years of proven results.', 'headline_sentiment': {'postive': '0.25946614146232605', 'negative': '0.3799891471862793', 'neutral': '0.25364914536476135'}, 'summary_sentiment': {'postive': '0.2572893500328064', 'negative': '0.37214919924736023', 'neutral': '0.2266107201576233'}, 'headline_label_name': 'negative', 'summary_label_name': 'negative', 'headline_label': 0, 'summary_label': 0}
{'postive': '0.25946614146232605', 'negative': '0.3799891471862793', 'neutral': '0.25364914536476135'}
{'postive': '0.2572893500328064', 'negative': '0.37214919924736023', 'neutral': '0.2266107201576233'}
0
0


In [ ]:
import pandas as pd
df = pd.DataFrame(dataset['default'])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 316086 entries, 0 to 316085
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   headline             316086 non-null  object
 1   summary              316086 non-null  object
 2   headline_sentiment   316086 non-null  object
 3   summary_sentiment    316086 non-null  object
 4   headline_label_name  316086 non-null  object
 5   summary_label_name   316086 non-null  object
 6   headline_label       316086 non-null  int64 
 7   summary_label        316086 non-null  int64 
dtypes: int64(2), object(6)
memory usage: 19.3+ MB


In [ ]:
df.pivot_table(index='headline_label_name', columns='summary_label_name', aggfunc='size', fill_value=0)

summary_label_name,negative,neutral,postive
headline_label_name,,,
negative,297919,3,12337
neutral,72,0,1
postive,5730,0,24


Acoording to pivot table `headline_label_name` vs `summary_label_name`,
- Majority class is Negative 303,721 (96.09%) for `summary_label_name` and 310,259 (98.16%) for `headline_label_name`.
- There are 12,337 (3.90%) article that have Negative headline but Positive summary.
- There are 5,730 (1.81%) article that have Positive headline but Negative summary label.

Insight:
- Dataset class is extremely **imbalance** (Negative majority), will use down sampling for majority class
- Neutral class is to small (<100 datapoints), we will drop this class

In [ ]:
df_trim = df[df['summary_label_name'] != 'neutral']

In [ ]:
df_trim['summary_label_name'].value_counts()

,count
summary_label_name,
negative,303721
postive,12362


In [ ]:
# df_trim.to_parquet('df_trim.parquet')

In [ ]:
df_trim[['summary','summary_label_name','summary_label']].drop_duplicates()['summary_label_name'].value_counts() #316083 --[drop duplicates]--> 159668

,count
summary_label_name,
negative,159534
postive,134


Since distribution of summary_label (after removing duplicates) is very imbalance, so we will move to check the headlines data.

In [ ]:
df_headline = df[df['headline_label_name']!='neutral'][['headline','headline_label_name','headline_label']]

In [ ]:
df_headline[['headline','headline_label_name']].drop_duplicates()['headline_label_name'].value_counts()

,count
headline_label_name,
negative,310068
postive,5748


Great, it is not bad for the distribution. We can do downsampling for `negative` class.

In [ ]:
# df_headline.to_parquet('df_headline.parquet')

In [ ]:
import random

idx_pos = list(df_headline[df_headline['headline_label_name']=='postive'].index)
idx_neg = list(df_headline[df_headline['headline_label_name']=='negative'].index)
selected_idx_neg = random.choices(idx_neg,k=len(idx_pos))

In [ ]:
df_headline_downsampled = df_headline.loc[selected_idx_neg+idx_pos]

In [ ]:
df_headline_downsampled.headline_label_name.value_counts()

,count
headline_label_name,
negative,5754
postive,5754


In [ ]:
df_headline_downsampled = df_headline_downsampled.reset_index(drop=True)

In [99]:
import pandas as pd

# EXPORT TO PARQUET
df_headline_downsampled.to_parquet('df_headline_downsampled.parquet')

# READ DATA PARQUET
df_headline_downsampled = pd.read_parquet('df_headline_downsampled.parquet')
df_headline_downsampled.head()

,headline,headline_label_name,headline_label
0,"TSMC Q1 profit inches past estimates, but grow...",negative,0
1,T. Rowe Price Group's (NASDAQ:TROW) investors ...,negative,0
2,XPeng's stock rockets after Volkswagen invests...,negative,0
3,"Hoth Therapeutics, Algorithm Sciences Scrap Re...",negative,0
4,Catalyst in licensing deal with Santhera for D...,negative,0


1825

In [100]:
import random
from sklearn.model_selection import train_test_split

random_state = random.randint(1,9999)
print(f"random_state: {random_state}")

X = df_headline_downsampled['headline']
y = df_headline_downsampled['headline_label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=random_state)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.65, random_state=random_state)

random_state: 1825


In [101]:
ds_train = X_train.copy()
ds_train = pd.concat([ds_train,y_train], axis=1)

ds_test = X_test.copy()
ds_test = pd.concat([ds_test,y_test], axis=1)

ds_val = X_val.copy()
ds_val = pd.concat([ds_val,y_val], axis=1)

In [102]:
from datasets import Dataset, DatasetDict
dataset_headline_train = Dataset.from_pandas(ds_train)
dataset_headline_test = Dataset.from_pandas(ds_test)
dataset_headline_val = Dataset.from_pandas(ds_val)

In [103]:
dataset_headline = DatasetDict({'train': Dataset.from_pandas(ds_train),
                                 'test': Dataset.from_pandas(ds_test),
                                 'valid': Dataset.from_pandas(ds_val),})

In [104]:
dataset_headline['train'][0]

{'headline': 'Frontier Communications backs FY23 adjusted EBITDA view $2.11B-$2.16B',
 'headline_label': 1,
 '__index_level_0__': 6590}

In [105]:
dataset_headline = dataset_headline.remove_columns(["__index_level_0__"])

In [106]:
def rename_colname(example):
  example['text'] = example.pop('headline')
  example['label'] = example.pop('headline_label')
  return example


dataset_headline = dataset_headline.map(rename_colname,num_proc=4)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/8055 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1208 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2245 [00:00<?, ? examples/s]

In [107]:
dataset_headline

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8055
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1208
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 2245
    })
})

## Fine Tune DistillBERT

In [119]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [120]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)

tok_train = dataset_headline['train'].map(preprocess_function, batched=True)
tok_test = dataset_headline['test'].map(preprocess_function, batched=True)
tok_valid = dataset_headline['valid'].map(preprocess_function, batched=True)

Map:   0%|          | 0/8055 [00:00<?, ? examples/s]

Map:   0%|          | 0/1208 [00:00<?, ? examples/s]

Map:   0%|          | 0/2245 [00:00<?, ? examples/s]

In [60]:
tok_train

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 8055
})

In [121]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [122]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2, trust_remote_code=True)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [123]:
import numpy as np
from datasets import load_metric

def compute_metrics(eval_pred):
   load_accuracy = load_metric("accuracy")
   load_f1 = load_metric("f1")

   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels, average='macro')["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [124]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

repo_name = "./headlines_news_sentiment_distil"

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=5,
   weight_decay=0.01,
   evaluation_strategy='epoch',
   save_strategy="epoch",
   metric_for_best_model="accuracy",
   push_to_hub=True,
   load_best_model_at_end=True,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tok_train,
   eval_dataset=tok_valid,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
   callbacks = [EarlyStoppingCallback(early_stopping_patience = 1)]
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [66]:
tok_valid

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask'],
    num_rows: 2418
})

In [125]:
# SEE THE MODEL PERFORMANCE BEFORE FINE TUNE
trainer.evaluate(tok_test)

{'eval_loss': 0.6926647424697876,
 'eval_model_preparation_time': 0.0026,
 'eval_accuracy': 0.49337748344370863,
 'eval_f1': 0.4904218362282879,
 'eval_runtime': 2.9157,
 'eval_samples_per_second': 414.315,
 'eval_steps_per_second': 26.066}

In [126]:
# TRAIN
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy,F1
1,0.483400,0.403494,0.002600,0.815590,0.815555
2,0.340200,0.398711,0.002600,0.834298,0.834267
3,0.234300,0.451350,0.002600,0.839198,0.839142
4,0.160400,0.544267,0.002600,0.839644,0.839638
5,0.115100,0.626466,0.002600,0.842316,0.842312


TrainOutput(global_step=2520, training_loss=0.26520579003152395, metrics={'train_runtime': 237.0292, 'train_samples_per_second': 169.916, 'train_steps_per_second': 10.632, 'total_flos': 300711085239252.0, 'train_loss': 0.26520579003152395, 'epoch': 5.0})

In [129]:
# SEE THE MODEL PERFORMANCE AFTER FINE TUNE
trainer.evaluate(tok_test)

{'eval_loss': 0.7081567645072937,
 'eval_model_preparation_time': 0.0026,
 'eval_accuracy': 0.8269867549668874,
 'eval_f1': 0.8267016184775096,
 'eval_runtime': 3.2711,
 'eval_samples_per_second': 369.296,
 'eval_steps_per_second': 23.234,
 'epoch': 5.0}

Great, we have improved our model performance after fine tune the model. Before we got F1 0.49 and Accuracy 0.49, after fine tune we got F1 0.82 adn Accuracy 0.82.

In [127]:
# SAVE MODEL TO HUGGING FACE REPOSITORY
trainer.push_to_hub("headlines_news_sentiment_distil")

events.out.tfevents.1726028572.c9e2a75fea34.8923.15:   0%|          | 0.00/8.84k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ishaq101/headlines_news_sentiment_distil/commit/e10a031a7219101731db471f837eefb74b49cef0', commit_message='headlines_news_sentiment_distil', commit_description='', oid='e10a031a7219101731db471f837eefb74b49cef0', pr_url=None, pr_revision=None, pr_num=None)

See the repository here [ishaq101/headlines_news_sentiment_distil](https://huggingface.co/ishaq101/headlines_news_sentiment_distil)

In [128]:
# TRY INFERENCING MODEL
from transformers import pipeline

pipe = pipeline("text-classification", model="ishaq101/headlines_news_sentiment_distil")

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [134]:
df_headline_downsampled.loc[321]

,321
headline,FTC Seeks to Block Amgen's Acquisition of Hori...
headline_label_name,negative
headline_label,0


In [135]:
df_headline_downsampled.loc[321]['headline']

"FTC Seeks to Block Amgen's Acquisition of Horizon Therapeutics: What's Next for Both Stocks?"

In [138]:
import time

st = time.time()
result = pipe("FTC Seeks to Block Amgen's Acquisition of Horizon Therapeutics: What's Next for Both Stocks?")
print(result)
print(f"runtime: {time.time() - st}s")
# 'LABEL_0':'negative', 'LABEL_1':'positive'

[{'label': 'LABEL_0', 'score': 0.999135434627533}]
runtime: 0.23390412330627441s
